In [50]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler
from pyspark.ml.feature import StopWordsRemover, Word2Vec, RegexTokenizer, Tokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import Row
from pyspark.streaming.kafka import KafkaUtils
import pyspark.sql.functions as f
import json
import re
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
import sys
from pyspark.sql.types import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import from_json
from pyspark.sql.functions import lit

import pickle
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from numpy import zeros
from keras.models import model_from_json

In [51]:
sc = spark.sparkContext
sql_context = SQLContext(sc)

In [52]:
sc

<SparkContext master=yarn appName=PySparkShell>

In [53]:
spark

In [54]:
sql_context

In [55]:
csi_pos_neg = spark.read.csv("hdfs:///user/spark/datafile/csiposneg.csv", header=True)

In [7]:
ddd = spark.read.csv("hdfs:///user/spark/datafile/fourth_test.csv", header=True)

In [10]:
csi_pos_neg.show(5)

+---+--------------------+-----+
|_c0|                text|label|
+---+--------------------+-----+
|  0|threatmeter hacke...|    0|
|  1|first android mal...|    0|
|  2|adobe fixes six c...|    0|
|  3| scienceporn  in ...|    0|
|  4|riskware hmoyfzb ...|    0|
+---+--------------------+-----+
only showing top 5 rows



In [11]:
ddd.show(5)

+---+----------+--------------------+-----+
|_c0|Unnamed: 0|                text|label|
+---+----------+--------------------+-----+
|  0|    332843| osxreverser  ove...|    0|
|  1|    292748|hackerfanatic yom...|    0|
|  2|   1024662|cve nginx http pr...|    0|
|  3|   2645604|jvndboracle mysql...|    0|
|  4|   1831637|rt geeknik  monoc...|    0|
+---+----------+--------------------+-----+
only showing top 5 rows



In [45]:
ddd.groupBy('label').count().orderBy('count').show()

+-----+-----+
|label|count|
+-----+-----+
|    0|49878|
|    1|49878|
+-----+-----+



In [56]:
a1 = ddd.collect()[24939:49878]
a2 = ddd.collect()[49878:74817]

a1 = spark.createDataFrame(a1)
a2 = spark.createDataFrame(a2)

NameError: name 'ddd' is not defined

In [57]:
a1 = a1.drop('_c0').drop('Unnamed: 0')
a1.show(5, truncate=False)

NameError: name 'a1' is not defined

In [23]:
a2 = a2.drop('_c0').drop('Unnamed: 0')
a2.show(5, truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------+-----+
|text                                                                                                                                  |label|
+--------------------------------------------------------------------------------------------------------------------------------------+-----+
|for whom the baja blasts                                                                                                              |1    |
|its scary how accurate that thing about girls being upset and then coming online and finding a tweet that describes their situation is|1    |
|this is a peaceful rally the sole purpose of the crowd is to demonstrate how angry people are towards the national securi             |1    |
|me im so forgiving                                                                                                                    |1    |

In [24]:
f4 = a1.union(a2)
f4.show(5)

+--------------------+-----+
|                text|label|
+--------------------+-----+
|operation emmenta...|    0|
|credit card numbe...|    0|
|tor browser app i...|    0|
|people follow lea...|    0|
|huawei reports  p...|    0|
+--------------------+-----+
only showing top 5 rows



In [25]:
f4.groupBy('label').count().orderBy('count').show()

+-----+-----+
|label|count|
+-----+-----+
|    0|24939|
|    1|24939|
+-----+-----+



In [57]:
f4.show(5)

+--------------------+-----+
|                text|label|
+--------------------+-----+
|operation emmenta...|    0|
|credit card numbe...|    0|
|tor browser app i...|    0|
|people follow lea...|    0|
|huawei reports  p...|    0|
+--------------------+-----+
only showing top 5 rows



In [26]:
csi_pos_neg.show(5)

+---+--------------------+-----+
|_c0|                text|label|
+---+--------------------+-----+
|  0|threatmeter hacke...|    0|
|  1|first android mal...|    0|
|  2|adobe fixes six c...|    0|
|  3| scienceporn  in ...|    0|
|  4|riskware hmoyfzb ...|    0|
+---+--------------------+-----+
only showing top 5 rows



In [7]:
csi_pos_neg.count()

199512

In [10]:
csi_pos_neg.groupBy('label').count().orderBy('count').show()

+-----+-----+
|label|count|
+-----+-----+
|    0|99756|
|    1|99756|
+-----+-----+



In [8]:
csi_pos_neg = csi_pos_neg.orderBy(rand())

In [53]:
csi_pos_neg.show(5)

+--------------------+-----+
|                text|label|
+--------------------+-----+
|windows xp endofl...|    0|
|unite against rac...|    1|
|dropbox files pet...|    0|
|hdp for retailers...|    0|
|meal prep doesnt ...|    1|
+--------------------+-----+
only showing top 5 rows



In [9]:
csi_pos_neg = csi_pos_neg.drop('_c0')

In [58]:
t1 = csi_pos_neg.collect()[:24939]
t2 = csi_pos_neg.collect()[24939:49878]
t3 = csi_pos_neg.collect()[49878:74817]
t4 = csi_pos_neg.collect()[74817:99756]

t5 = csi_pos_neg.collect()[99756:124695]
t6 = csi_pos_neg.collect()[124695:149634]
t7 = csi_pos_neg.collect()[149634:174573]
t8 = csi_pos_neg.collect()[174573:199512]

In [59]:
t1 = spark.createDataFrame(t1)
t2 = spark.createDataFrame(t2)
t3 = spark.createDataFrame(t3)
t4 = spark.createDataFrame(t4)

t5 = spark.createDataFrame(t5)
t6 = spark.createDataFrame(t6)
t7 = spark.createDataFrame(t7)
t8 = spark.createDataFrame(t8)

In [60]:
tㅇ.groupBy('label').count().orderBy('count').show()

NameError: name 'tᄋ' is not defined

In [31]:
t8.groupBy('label').count().orderBy('count').show()

+-----+-----+
|label|count|
+-----+-----+
|    1|24939|
+-----+-----+



In [ ]:
first_train=t1.union(t5) #word_counts_1 t1
first_test=t2.union(t6)
second_train=first_train.union(first_test) #word_counts_1 t1, t2
second_test=t3.union(t7)
third_train=second_train.union(second_test)
third_test=t4.union(t8)
fourth_train=third_train.union(third_test)
#fourth_test

In [13]:
fourth_train.show(5)

+--------------------+-----+
|                text|label|
+--------------------+-----+
|threatmeter hacke...|    1|
|first android mal...|    1|
|adobe fixes six c...|    1|
| scienceporn  in ...|    1|
|riskware hmoyfzb ...|    1|
+--------------------+-----+
only showing top 5 rows



In [34]:
fourth_train.count()

199512

In [38]:
csi_pos_neg.groupBy('label').count().orderBy('count').show()

+-----+-----+
|label|count|
+-----+-----+
|    0|99756|
|    1|99756|
+-----+-----+



In [14]:
fourth_train = fourth_train.drop('_c0')

In [15]:
fourth_test = fourth_train.union(f4)

NameError: name 'f4' is not defined

In [16]:
fourth_test.count()

NameError: name 'fourth_test' is not defined

In [83]:
second_test = second_train.union(second_test)

In [152]:
third_test = third_train.union(third_test)

In [18]:
first_train.groupBy('label').count().orderBy('count').show()

+-----+-----+
|label|count|
+-----+-----+
|    0|24939|
|    1|24939|
+-----+-----+



In [19]:
first_test.groupBy('label').count().orderBy('count').show()

+-----+-----+
|label|count|
+-----+-----+
|    1|24939|
|    0|24939|
+-----+-----+



In [20]:
second_train.groupBy('label').count().orderBy('count').show()

+-----+-----+
|label|count|
+-----+-----+
|    0|49878|
|    1|49878|
+-----+-----+



In [84]:
second_test.groupBy('label').count().orderBy('count').show()

+-----+-----+
|label|count|
+-----+-----+
|    0|74817|
|    1|74817|
+-----+-----+



In [85]:
second_test.count()

149634

In [22]:
third_train.groupBy('label').count().orderBy('count').show()

+-----+-----+
|label|count|
+-----+-----+
|    0|74817|
|    1|74817|
+-----+-----+



In [153]:
third_test.groupBy('label').count().orderBy('count').show()

+-----+-----+
|label|count|
+-----+-----+
|    0|99756|
|    1|99756|
+-----+-----+



In [182]:
fourth_train.groupBy('label').count().orderBy('count').show()

+-----+-----+
|label|count|
+-----+-----+
|    0|99756|
|    1|99756|
+-----+-----+



In [211]:
fourth_test.groupBy('label').count().orderBy('count').show()

+-----+------+
|label| count|
+-----+------+
|    0|149634|
|    1|149634|
+-----+------+



In [10]:
# df1 = csi_pos_neg.collect()[:49878]
# df2 = csi_pos_neg.collect()[49878:99756]
# df3 = csi_pos_neg.collect()[99756:149634]
# df4 = csi_pos_neg.collect()[149634:199512]

In [11]:
# df1 = spark.createDataFrame(df1)
# df2 = spark.createDataFrame(df2)
# df3 = spark.createDataFrame(df3)
# df4 = spark.createDataFrame(df4)

In [9]:
csi_pos_neg.rdd.getNumPartitions()

5

In [9]:
# csi_pos_neg = csi_pos_neg.repartition(10000)

In [10]:
# csi_pos_neg.rdd.getNumPartitions()

10000

In [ ]:
with open('/root/word_counts1.json') as json_file:
    json_data = json.load(json_file)

word_counts_1 = {}
for i in json_data:
    word_counts_1.update({i[0]:i[1]})

In [41]:
word_counts_1

In [19]:
word_counts_1 = sc.broadcast(word_counts_1)

In [ ]:
with open('/root/word_counts2.json') as json_file:
    json_data = json.load(json_file)

word_counts_2 = {}
for i in json_data:
    word_counts_2.update({i[0]:i[1]})
word_counts_2

In [21]:
word_counts_2 = sc.broadcast(word_counts_2)

In [22]:
# https://www.programmerall.com/article/89051746485/ -> broadcast 변수 업데이트

In [ ]:
# get_tokens 함수과 같은 용도
stage_1 = RegexTokenizer(inputCol= 'text', outputCol='pos_t', pattern= '\\W')

In [ ]:
r1 = stage_1.transform(first_train)
r1.show(5)

In [ ]:
# get_lemma 함수와 같은 용도
stage_2 = StopWordsRemover(inputCol= stage_1.getOutputCol(), outputCol= 'filtered_words')

In [ ]:
r2 = stage_2.transform(r1)
r2.show(5)

In [ ]:
r2.rdd.getNumPartitions()

In [30]:
r2.count()

49878

In [ ]:
########################################################################

In [31]:
class BroadcastWrapper(object):
    def __init__(self, data, token_list):
        self.broadcast_var = data
#         self.last_updated_time = datetime.now()
        self.token_list = token_list
    
#     def is_should_be_updated(self, data):
#         cur_time = datetime.now()
#         diff_sec = (cur_time - self.last_updated_time).total_seconds()
#         return self.broadcast_var is None or diff_sec> 1
    
    def update_and_get_data(self, spark):
        a = self.broadcast_var.value
        self.broadcast_var.unpersist()
        for i in self.token_list:
            a[i] += 1
        new_data = a
        self.broadcast_var = spark.broadcast(new_data)
#         self.last_updated_time = datetime.now()
#         return len(self.token_list)
        return self.broadcast_var

In [32]:
def nlkt_analysis(text):
    broadcast_wrapper = BroadcastWrapper(word_counts_1, text)
    length = broadcast_wrapper.update_and_get_data(sc)
    return length
    return new_data.value


In [33]:
udf_nlkt = udf(nlkt_analysis, StringType())    
new_df = r2.withColumn("temp", udf_nlkt("filtered_words"))

Traceback (most recent call last):
  File "/root/spark/python/pyspark/serializers.py", line 597, in dumps
    return cloudpickle.dumps(obj, 2)
  File "/root/spark/python/pyspark/cloudpickle.py", line 863, in dumps
    cp.dump(obj)
  File "/root/spark/python/pyspark/cloudpickle.py", line 260, in dump
    return Pickler.dump(self, obj)
  File "/usr/lib/python3.6/pickle.py", line 409, in dump
    self.save(obj)
  File "/usr/lib/python3.6/pickle.py", line 476, in save
    f(self, obj) # Call unbound method with explicit self
  File "/usr/lib/python3.6/pickle.py", line 736, in save_tuple
    save(element)
  File "/usr/lib/python3.6/pickle.py", line 476, in save
    f(self, obj) # Call unbound method with explicit self
  File "/root/spark/python/pyspark/cloudpickle.py", line 400, in save_function
    self.save_function_tuple(obj)
  File "/root/spark/python/pyspark/cloudpickle.py", line 549, in save_function_tuple
    save(state)
  File "/usr/lib/python3.6/pickle.py", line 476, in save
    f(

PicklingError: Could not serialize object: Exception: It appears that you are attempting to reference SparkContext from a broadcast variable, action, or transformation. SparkContext can only be used on the driver, not in code that it run on workers. For more information, see SPARK-5063.

In [ ]:
#######################################################################

In [ ]:
####################################################################

In [36]:
from pyspark.ml.feature import CountVectorizer
stage_3 = CountVectorizer(inputCol=stage_2.getOutputCol(), outputCol='word1', vocabSize=50, minDF=2.0)

In [37]:
r3 = stage_3.fit(r2).transform(r2)
r3.show(5)

+--------------------+-----+--------------------+--------------------+---------------+
|                text|label|               pos_t|      filtered_words|          word1|
+--------------------+-----+--------------------+--------------------+---------------+
|threatmeter hacke...|    1|[threatmeter, hac...|[threatmeter, hac...| (50,[4],[1.0])|
|first android mal...|    1|[first, android, ...|[first, android, ...|(50,[35],[1.0])|
|adobe fixes six c...|    1|[adobe, fixes, si...|[adobe, fixes, si...|     (50,[],[])|
| scienceporn  in ...|    1|[scienceporn, in,...|[scienceporn, vac...|     (50,[],[])|
|riskware hmoyfzb ...|    1|[riskware, hmoyfz...|[riskware, hmoyfz...|     (50,[],[])|
+--------------------+-----+--------------------+--------------------+---------------+
only showing top 5 rows



In [22]:
r3.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)
 |-- pos_t: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- filtered_words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- word1: vector (nullable = true)



In [ ]:
########################################################################

In [15]:
#CUSTOM TRANSFORMER

In [61]:
df = r2.select("filtered_words")
df.show(5)

+--------------------+
|      filtered_words|
+--------------------+
|[threatmeter, hac...|
|[first, android, ...|
|[adobe, fixes, si...|
|[scienceporn, vac...|
|[riskware, hmoyfz...|
+--------------------+
only showing top 5 rows



In [67]:
pos_t = Tokenizer()
neg_t = Tokenizer()

In [68]:
pos_vocab = [w for i, w in enumerate(word_counts_1) if i < 5000]
neg_vocab = [w for i, w in enumerate(word_counts_2) if i < 5000]

In [69]:
pos_t.fit_on_texts(pos_vocab)
neg_t.fit_on_texts(neg_vocab)

In [70]:
encoded_docs_pos = pos_t.texts_to_sequences(df.toPandas())

In [72]:
df.toPandas().columns

Index(['filtered_words'], dtype='object')

In [73]:
df.toPandas()

,filtered_words
0,"[threatmeter, hacked, emails, san, francisco, ..."
1,"[first, android, malware, targeting, pcs, unco..."
2,"[adobe, fixes, six, code, execution, bugs, fla..."
3,"[scienceporn, vacuum, guess]"
4,"[riskware, hmoyfzb, fbjaz]"
...,...
49873,"[hi, lil, monster, lives, super, ridecheers, i..."
49874,"[topleftbrick, veronica, mikeyil, totally, fai..."
49875,"[lovely, day, golf, bob, schulz, scott, pemric..."
49876,"[manufacturing, job, might, great, fit, click,..."


In [66]:
type(df.toPandas())

pandas.core.frame.DataFrame

In [26]:
word_counts_1.value['allow']

105727

In [52]:
'allow' not in word_counts_1.value.keys()

False

In [69]:
class BroadcastWrapper(object):
    def __init__(self, data, token_list):
        self.broadcast_var = sc.broadcast(data)
#         self.last_updated_time = datetime.now()
        self.token_list = token_list
    
#     def is_should_be_updated(self, data):
#         cur_time = datetime.now()
#         diff_sec = (cur_time - self.last_updated_time).total_seconds()
#         return self.broadcast_var is None or diff_sec> 1
    
    def update_and_get_data(self, spark):
        a = self.broadcast_var.value
        self.broadcast_var.unpersist()
        for i in self.token_list:
            for j in i:
                if j not in a.keys():
                    a[j] = 1
                else:
                    a[j] += 1
        new_data = a
        self.broadcast_var = spark.broadcast(new_data)
#         self.last_updated_time = datetime.now()
#         return len(self.token_list)
        return self.broadcast_var

In [24]:
## dynamic word ranking 고려x

In [56]:
from pyspark.ml import Transformer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
from typing import Dict
from pyspark.sql import DataFrame

class TextToSequence(Transformer):
    
    pos_t = Tokenizer()
    neg_t = Tokenizer()
    pos_vocab = []
    neg_vocab = []
    
    def __init__(self, w1: Dict[str, int], w2:Dict[str, int]):
        super(TextToSequence, self).__init__()

        self.pos_vocab = [w for i, w in enumerate(w1) if i < 5000]
        self.neg_vocab = [w for i, w in enumerate(w2) if i < 5000]
    
    
    def _transform(self, df: DataFrame):
        df = df.toPandas()
        X = []
        self.pos_t.fit_on_texts(self.pos_vocab)
        self.neg_t.fit_on_texts(self.neg_vocab)
        print(self.pos_vocab)
#         encoded_docs_pos = self.pos_t.texts_to_sequences(df.select("filtered_words1").toPandas())
#         encoded_docs_neg = self.neg_t.texts_to_sequences(df.select("filtered_words1").toPandas())
        encoded_docs_pos = self.pos_t.texts_to_sequences(df["filtered_words"])
        encoded_docs_neg = self.neg_t.texts_to_sequences(df["filtered_words"])

        X_p = pad_sequences(encoded_docs_pos, maxlen=100, padding='post')
        X_n = pad_sequences(encoded_docs_neg, maxlen=100, padding='post')

        df = df.assign(feature1 = X_p.tolist())
        df = df.assign(feature2 = X_n.tolist())
        
#         cols = ['feature1','feature2']
#         def feature_add(x) :
#             tmp = []
#             tmp.append(x.values)
#             return tmp[0].tolist()
        
#         df.insert(0, 'features', df[cols].apply(feature_add, axis=1))
        
        fdf = spark.createDataFrame(df)
        return fdf

In [ ]:
# dynmaic word ranking 고려o

# Train용

In [70]:
r2.count()

249390

In [22]:
from pyspark.ml import Transformer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
from typing import Dict
from pyspark.sql import DataFrame

class TextToSequence(Transformer):
    w1 = dict()
    w2 = dict()
    pos_t = Tokenizer()
    neg_t = Tokenizer()
    pos_vocab = []
    neg_vocab = []
    
    def __init__(self, w1: Dict[str, int], w2:Dict[str, int]):
        super(TextToSequence, self).__init__()
        self.w1 = w1
        self.w2 = w2
        
    
    def _transform(self, df: DataFrame):
        df = df.toPandas()
        pos_df = df[:99756]
        neg_df = df[99756:]
        
        broadcast_wrapper1 = BroadcastWrapper(self.w1, pos_df["filtered_words"])
        ww1 = broadcast_wrapper1.update_and_get_data(sc).value
        
        broadcast_wrapper2 = BroadcastWrapper(self.w2, neg_df["filtered_words"])
        ww2 = broadcast_wrapper2.update_and_get_data(sc).value
        
        www1 = {k: v for k, v in sorted(ww1.items(), key=lambda item: item[1], reverse=True)}
        www2 = {k: v for k, v in sorted(ww2.items(), key=lambda item: item[1], reverse=True)}
        
        aa = [w for i, w in enumerate(www1) if i < 5000]
        bb = [w for i, w in enumerate(www2) if i < 5000]
        
        self.pos_t.fit_on_texts(aa)
        self.neg_t.fit_on_texts(bb)
        print(aa)
#         encoded_docs_pos = self.pos_t.texts_to_sequences(df.select("filtered_words1").toPandas())
#         encoded_docs_neg = self.neg_t.texts_to_sequences(df.select("filtered_words1").toPandas())
        encoded_docs_pos = self.pos_t.texts_to_sequences(df["filtered_words"])
        encoded_docs_neg = self.neg_t.texts_to_sequences(df["filtered_words"])
        print(encoded_docs_pos[:10])
        X_p = pad_sequences(encoded_docs_pos, maxlen=100, padding='post')
        X_n = pad_sequences(encoded_docs_neg, maxlen=100, padding='post')

        df = df.assign(feature1 = X_p.tolist())
        df = df.assign(feature2 = X_n.tolist())
        
#         cols = ['feature1','feature2']
#         def feature_add(x) :
#             tmp = []
#             tmp.append(x.values)
#             return tmp[0].tolist()
        
#         df.insert(0, 'features', df[cols].apply(feature_add, axis=1))
        
        fdf = spark.createDataFrame(df)
        return fdf

# TEST용

In [71]:
from pyspark.ml import Transformer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
from typing import Dict
from pyspark.sql import DataFrame

class TextToSequence(Transformer):
    w1 = dict()
    w2 = dict()
    pos_t = Tokenizer()
    neg_t = Tokenizer()
    pos_vocab = []
    neg_vocab = []
    
    def __init__(self, w1: Dict[str, int], w2:Dict[str, int]):
        super(TextToSequence, self).__init__()
        self.w1 = w1
        self.w2 = w2
        
    
    def _transform(self, df: DataFrame):
        df = df.toPandas()
        cut_df = df[:199512]
        df = df[199512:]
        # csi_pos인 filtered_words에 대해서만...
        broadcast_wrapper1 = BroadcastWrapper(self.w1, cut_df["filtered_words"][:99756])
        ww1 = broadcast_wrapper1.update_and_get_data(sc).value
        
        broadcast_wrapper2 = BroadcastWrapper(self.w2, cut_df["filtered_words"][99756:])
        ww2 = broadcast_wrapper2.update_and_get_data(sc).value
        
        www1 = {k: v for k, v in sorted(ww1.items(), key=lambda item: item[1], reverse=True)}
        www2 = {k: v for k, v in sorted(ww2.items(), key=lambda item: item[1], reverse=True)}
        
        aa = [w for i, w in enumerate(www1) if i < 5000]
        bb = [w for i, w in enumerate(www2) if i < 5000]
        
        self.pos_t.fit_on_texts(aa)
        self.neg_t.fit_on_texts(bb)
        print(aa)
#         encoded_docs_pos = self.pos_t.texts_to_sequences(df.select("filtered_words1").toPandas())
#         encoded_docs_neg = self.neg_t.texts_to_sequences(df.select("filtered_words1").toPandas())
        encoded_docs_pos = self.pos_t.texts_to_sequences(df["filtered_words"])
        encoded_docs_neg = self.neg_t.texts_to_sequences(df["filtered_words"])
        print(encoded_docs_pos[:10])
        X_p = pad_sequences(encoded_docs_pos, maxlen=100, padding='post')
        X_n = pad_sequences(encoded_docs_neg, maxlen=100, padding='post')

        df = df.assign(feature1 = X_p.tolist())
        df = df.assign(feature2 = X_n.tolist())
        
#         cols = ['feature1','feature2']
#         def feature_add(x) :
#             tmp = []
#             tmp.append(x.values)
#             return tmp[0].tolist()
        
#         df.insert(0, 'features', df[cols].apply(feature_add, axis=1))
        
        fdf = spark.createDataFrame(df)
        return fdf

In [125]:
r2.select("filtered_words").show(1, truncate=False)

+-----------------------------------------------------------------+
|filtered_words                                                   |
+-----------------------------------------------------------------+
|[blacklivesmatter, blm, ldn, blmldn, ldnblm, us, embassy, london]|
+-----------------------------------------------------------------+
only showing top 1 row



In [121]:
asdfasdf = r2.toPandas()
asdfasdf["filtered_words"][:-49878]

0        [want, land, job, like, sales, representative,...
1        [fitness, friends, like, gold, rare, valuable,...
2        [tradecalls, nomura, maintains, palo, alto, ne...
3        [job, might, great, fit, rn, staff, skilled, n...
4                   [im, seafood, junction, oak, park, il]
                               ...                        
49873    [heres, challenge, teacher, friends, students,...
49874    [greatest, shot, tallest, hill, behind, ive, a...
49875    [nothing, musical, sunset, claude, debussy, ca...
49876    [ipv, focus, month, warm, fuzzy, side, ipv, tu...
49877    [big, data, minute, making, sense, new, natura...
Name: filtered_words, Length: 49878, dtype: object

In [122]:
asdfasdf["filtered_words"]

0        [want, land, job, like, sales, representative,...
1        [fitness, friends, like, gold, rare, valuable,...
2        [tradecalls, nomura, maintains, palo, alto, ne...
3        [job, might, great, fit, rn, staff, skilled, n...
4                   [im, seafood, junction, oak, park, il]
                               ...                        
99751    [disorderly, conduct, charge, handing, cardboa...
99752    [techcentral, third, data, protection, pros, a...
99753    [good, morning, montevideo, sunrise, noon, sun...
99754    [euleros, sp, dockerengine, eulerossa, tenable...
99755    [pret, chicken, caesar, amp, bacon, baguette, ...
Name: filtered_words, Length: 99756, dtype: object

In [72]:
# USE THE TRANSFORMER WITHOUT PIPELINE
text_sequence = TextToSequence(w1 = word_counts_1.value, w2 = word_counts_2.value)
df_example = text_sequence.transform(r2)

['allow', 'attacker', 'via', 'vulnerability', 'remote', 'arbitrary', 'cve', 'user', 'execute', 'file', 'code', 'service', 'script', 'cause', 'parameter', 'server', 'denial', 'craft', 'version', 'web', 'access', 'earlier', 'unspecified', 'vector', 'information', 'windows', 'affect', 'x', 'sql', 'aka', 'privilege', 'local', 'attack', 'command', 'oracle', 'data', 'could', 'application', 'crosssite', 'memory', 'function', 'overflow', 'security', 'issue', 'use', 'request', 'buffer', 'html', 'obtain', 'inject', 'multiple', 'system', 'crash', 'injection', 'id', 'authenticate', 'sp', 'component', 'android', 'read', 'relate', 'sensitive', 'microsoft', 'sd', 'impact', 'possibly', 'prior', 'directory', 'properly', 'unknown', 'exploit', 'php', 'network', 'execution', 'bypass', 'product', 'http', 'cisco', 'ibm', 'result', 'different', 'note', 'corruption', 'may', 'url', 'job', 'cybersecurity', 'successful', 'password', 'kernel', 'gain', 'exist', 'device', 'linux', 'discover', 'os', 'software', 'cer

[[638, 617, 1367, 514, 378, 718, 401, 3753], [1604, 968, 1433, 3720, 154, 36, 702, 2338], [1447, 338, 239, 117, 239, 154, 1562, 330, 316, 1447, 117], [479, 369, 3862, 2178, 2762, 2732, 2718, 1042, 465], [677, 625, 3556, 677, 3141, 4878, 210, 3140, 2907], [], [45, 3992, 4898, 746], [678, 762, 850, 104, 790, 134, 142, 2487, 134, 4102], [453, 827, 1924], [1603]]


In [73]:
df_example.count()

49878

In [57]:
len(word_counts_1.value)

134615

In [70]:
len(word_counts_2.value)

710505

In [46]:
df_example.select('feature1','feature2').show(5, truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|feature1                                                                                                                                                                                                                                                                                                                          |feature2                          

In [127]:
df_example.count()

99756

In [47]:
df_example.rdd.getNumPartitions()

20

In [129]:
df_example.printSchema()

root
 |-- text: string (nullable = true)
 |-- label: string (nullable = true)
 |-- pos_t: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- filtered_words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- feature1: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- feature2: array (nullable = true)
 |    |-- element: long (containsNull = true)



In [23]:
df_example.show(1, truncate=False)

+-----+-------------------------------------------------------------------------------------------------------------+-----+---------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|_c0  |text           

In [ ]:
# 리스트를 벡터화하는 1번째 방법

In [30]:
from pyspark.ml.feature import VectorAssembler
TEMPERATURE_COUNT = 100
assembler_exploded = VectorAssembler(
    inputCols=["feature1[{}]".format(i) for i in range(TEMPERATURE_COUNT)], 
    outputCol="f1_vector"
)
df_exploded = df_example.select(
    df_example["label"], 
    *[df_example["feature1"][i] for i in range(TEMPERATURE_COUNT)]
)
converted_df = assembler_exploded.transform(df_exploded)
final_df = converted_df.select("f1_vector", "label")

In [31]:
df_exploded.printSchema()

root
 |-- label: string (nullable = true)
 |-- feature1[0]: long (nullable = true)
 |-- feature1[1]: long (nullable = true)
 |-- feature1[2]: long (nullable = true)
 |-- feature1[3]: long (nullable = true)
 |-- feature1[4]: long (nullable = true)
 |-- feature1[5]: long (nullable = true)
 |-- feature1[6]: long (nullable = true)
 |-- feature1[7]: long (nullable = true)
 |-- feature1[8]: long (nullable = true)
 |-- feature1[9]: long (nullable = true)
 |-- feature1[10]: long (nullable = true)
 |-- feature1[11]: long (nullable = true)
 |-- feature1[12]: long (nullable = true)
 |-- feature1[13]: long (nullable = true)
 |-- feature1[14]: long (nullable = true)
 |-- feature1[15]: long (nullable = true)
 |-- feature1[16]: long (nullable = true)
 |-- feature1[17]: long (nullable = true)
 |-- feature1[18]: long (nullable = true)
 |-- feature1[19]: long (nullable = true)
 |-- feature1[20]: long (nullable = true)
 |-- feature1[21]: long (nullable = true)
 |-- feature1[22]: long (nullable = true)
 |

In [32]:
converted_df.printSchema()

root
 |-- label: string (nullable = true)
 |-- feature1[0]: long (nullable = true)
 |-- feature1[1]: long (nullable = true)
 |-- feature1[2]: long (nullable = true)
 |-- feature1[3]: long (nullable = true)
 |-- feature1[4]: long (nullable = true)
 |-- feature1[5]: long (nullable = true)
 |-- feature1[6]: long (nullable = true)
 |-- feature1[7]: long (nullable = true)
 |-- feature1[8]: long (nullable = true)
 |-- feature1[9]: long (nullable = true)
 |-- feature1[10]: long (nullable = true)
 |-- feature1[11]: long (nullable = true)
 |-- feature1[12]: long (nullable = true)
 |-- feature1[13]: long (nullable = true)
 |-- feature1[14]: long (nullable = true)
 |-- feature1[15]: long (nullable = true)
 |-- feature1[16]: long (nullable = true)
 |-- feature1[17]: long (nullable = true)
 |-- feature1[18]: long (nullable = true)
 |-- feature1[19]: long (nullable = true)
 |-- feature1[20]: long (nullable = true)
 |-- feature1[21]: long (nullable = true)
 |-- feature1[22]: long (nullable = true)
 |

In [33]:
final_df.printSchema()

root
 |-- f1_vector: vector (nullable = true)
 |-- label: string (nullable = true)



In [34]:
final_df.show(5, truncate=False)

+------------------------------------------------------------------------------+-----+
|f1_vector                                                                     |label|
+------------------------------------------------------------------------------+-----+
|(100,[0,1,2,3,4,5,6,7],[4303.0,866.0,1635.0,2954.0,1705.0,96.0,1471.0,2960.0])|1    |
|(100,[0,1,2,3],[1635.0,255.0,4643.0,1115.0])                                  |0    |
|(100,[0,1,2,3,4,5],[2242.0,2226.0,1136.0,2802.0,324.0,2448.0])                |0    |
|(100,[0,1,2],[866.0,174.0,4915.0])                                            |1    |
|(100,[0],[1849.0])                                                            |1    |
+------------------------------------------------------------------------------+-----+
only showing top 5 rows



In [ ]:
# 리스트를 벡터화하는 두번째 방법

In [74]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
df_with_vectors = df_example.select(
    df_example["label"], 
    list_to_vector_udf(df_example["feature1"]).alias("feature1"),
    list_to_vector_udf(df_example["feature2"]).alias("feature2")
)

In [75]:
df_with_vectors.rdd.getNumPartitions()

20

In [76]:
df_with_vectors.printSchema()

root
 |-- label: string (nullable = true)
 |-- feature1: vector (nullable = true)
 |-- feature2: vector (nullable = true)



In [38]:
df_with_vectors.select('label', 'feature1').limit(5).toPandas()

,label,feature1
0,1,"[4303.0, 866.0, 1635.0, 2954.0, 1705.0, 96.0, ..."
1,0,"[1635.0, 255.0, 4643.0, 1115.0, 0.0, 0.0, 0.0,..."
2,0,"[2242.0, 2226.0, 1136.0, 2802.0, 324.0, 2448.0..."
3,1,"[866.0, 174.0, 4915.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
4,1,"[1849.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."


In [80]:
# va = VectorAssembler(inputCols=['feature1'], outputCol='features')
# d1 = va.transform(df_with_vectors)

In [77]:
label_str_index = StringIndexer(inputCol='label', outputCol='label_index')
d4 = label_str_index.fit(df_with_vectors).transform(df_with_vectors)

In [78]:
d4.printSchema()

root
 |-- label: string (nullable = true)
 |-- feature1: vector (nullable = true)
 |-- feature2: vector (nullable = true)
 |-- label_index: double (nullable = false)



In [28]:
d4.rdd.getNumPartitions()

20

In [29]:
d4.select('label_index', 'feature1').limit(5).toPandas()

,label_index,feature1
0,0.0,"[805.0, 2640.0, 51.0, 811.0, 0.0, 0.0, 0.0, 0...."
1,0.0,"[285.0, 60.0, 1614.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,0.0,"[111.0, 3829.0, 11.0, 76.0, 405.0, 0.0, 0.0, 0..."
3,0.0,"[1524.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
4,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [43]:
d1.count()

49878

In [26]:
df_with_vectors.show(1, truncate=False)

+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|feature1                                                                                                                                                            

In [ ]:
# USE THE TRANSFORMER WITH PIPELINE

In [15]:
text_sequence = TextToSequence(w1 = word_counts_1, w2 = word_counts_2)
#df_example = text_sequence.transform(r2)

In [16]:
from pyspark.ml.feature import Bucketizer

text_sequence = TextToSequence(w1 = word_counts_1, w2 = word_counts_2)
med_model = Pipeline(stages=[stage_1, stage_2, text_sequence]).fit(csi_pos_neg)
bucketedData = med_model.transform(csi_pos_neg)
bucketedData.show(5)

+--------------------+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|            features|_c0|                text|label|               pos_t|      filtered_words|            feature1|            feature2|
+--------------------+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|[[2871, 52, 614, ...|  0|threatmeter hacke...|    0|[threatmeter, hac...|[threatmeter, hac...|[2871, 52, 614, 4...|[1693, 1001, 2716...|
|[[504, 59, 1094, ...|  1|first android mal...|    0|[first, android, ...|[first, android, ...|[504, 59, 1094, 0...|[46, 3724, 0, 0, ...|
|[[105, 4698, 11, ...|  2|adobe fixes six c...|    0|[adobe, fixes, si...|[adobe, fixes, si...|[105, 4698, 11, 7...|[1578, 667, 2657,...|
|[[1507, 0, 0, 0, ...|  3| scienceporn  in ...|    0|[scienceporn, in,...|[scienceporn, vac...|[1507, 0, 0, 0, 0...|[507, 0, 0, 0, 0,...|
|[[0, 0, 0, 0, 0, ...|  4|riskware

In [27]:
df_transform_fin = bucketedData.orderBy(rand())

In [28]:
df_transform_fin.rdd.getNumPartitions()

200

In [35]:
df_transform_fin = df_transform_fin.repartition(10000)
df_transform_fin.rdd.getNumPartitions()

10000

In [ ]:
nb_classes = df_transform_fin.select("label").distinct().count()
nb_classes

In [29]:
nb_classes = d4.select("label").distinct().count()
nb_classes

2

In [30]:
d4.count()

199512

In [37]:
sc = spark._jsc.sc() 
n_workers =  len([executor.host() for executor in sc.statusTracker().getExecutorInfos() ]) -1

print(n_workers)

0


In [31]:
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Input, Dense, concatenate
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers.embeddings import Embedding
from tensorflow.python.keras.layers import Conv1D, Flatten, GlobalMaxPooling1D, Dropout, MaxPooling1D, Activation, Bidirectional, Conv2D, GlobalMaxPooling2D

In [32]:
vocab_sizes = 5001
neg_vocab_sizes = 5001
embedding_vector_length = 100
hidden_dims = 250
max_length = 100

In [33]:
from tensorflow.keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(Embedding(vocab_sizes, embedding_vector_length, input_length=max_length))

#어휘 크기(최대 정수 인덱스+1, 고밀도 임베딩의 치수, 입력 시퀀스의 길이

# 모델은 크기의 정수 행렬 (배치,
# input_length) 및 입력에서 가장 큰 정수 (즉, 단어 인덱스)
#은 999 (어휘 크기)보다 크지 않아야합니다.
# 이제 model.output_shape는 (None, 10, 64)이고, 여기서`None`은 배치입니다.
# dimension.


# model.add(Conv1D(128, 5, activation='relu'))
model.add(Bidirectional(LSTM(128)))
# model.add(GlobalMaxPooling1D())
model.add(Activation('relu'))
model.add(Dense(nb_classes))
model.add(Activation('sigmoid'))
# model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# #체크포인트 생성?!
# # checkpoint_filepath = '/root/spark/baseline/'
# fname = "checkpoint-{epoch:02d}-{val_loss:.2f}"
# checkpoint = ModelCheckpoint(fname, monitor="val_loss", mode="min",
#     save_best_only=True, verbose=1)
# callbacks = [checkpoint]

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          500100    
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               234496    
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 514       
_________________________________________________________________
activation_1 (Activation)    (None, 2)                 0         
Total params: 735,110
Trainable params: 735,110
Non-trainable params: 0
_________________________________________________________________
None


In [129]:
# CSI positive Embedding model
inputA = Input(shape=(100,))
x = Embedding(vocab_sizes, embedding_vector_length, input_length=max_length)(inputA)
# x = Conv1D(128, 5, activation='relu')(x)
# x = MaxPool1D(strides=1, padding='valid')(x)
x = Bidirectional(LSTM(128))(x)
x = Dropout(0.2)(x)
x = Activation('relu')(x)
x = Model(inputs=inputA, outputs=x)

# CSI negative Embedding model
inputC = Input(shape=(100,))
k = Embedding(neg_vocab_sizes, embedding_vector_length, input_length=max_length)(inputC)
# k = Conv1D(128, 5, activation='relu')(k)
# k = MaxPool1D(strides=1, padding='valid')(k)
k = Bidirectional(LSTM(128))(k)
k = Dropout(0.2)(k)
k = Activation('relu')(k)
k = Model(inputs=inputC, outputs=k)

combined = concatenate([x.output, k.output], axis=1)
# z = Flatten()(combined)
z = Dense(8, activation='relu')(combined)
z = Dense(1, activation="sigmoid")(z)

model = Model(inputs=[x.input, k.input], outputs=z)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 100, 100)     500100      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 100, 100)     500100      input_2[0][0]                    
____________________________________________________________________________________________

In [35]:
from keras import optimizers, regularizers
from tensorflow.keras import optimizers

optimizer_conf = optimizers.Adam(lr=0.01)
opt_conf = optimizers.serialize(optimizer_conf)

In [36]:
from tensorflow.keras.models import model_from_json
from elephas import spark_model, ml_model
from elephas.ml_model import ElephasEstimator

estimator = ElephasEstimator()
estimator.setFeaturesCol("feature1")
estimator.setLabelCol("label_index")
estimator.set_keras_model_config(model.to_json())
estimator.set_categorical_labels(True) # dense 1이면 False로 설정해야함
estimator.set_nb_classes(nb_classes)
estimator.set_num_workers(1)
estimator.set_epochs(10)
estimator.set_batch_size(32)
estimator.set_verbosity(1)
estimator.set_validation_split(0.10)
estimator.set_optimizer_config(opt_conf)
estimator.set_mode("synchronous")
estimator.set_loss("binary_crossentropy")
estimator.set_metrics(['acc'])

ElephasEstimator_bb12150c039d

In [37]:
dl_pipeline = Pipeline(stages=[estimator])

In [38]:
my_dl = dl_pipeline.fit(d4)

>>> Fit model
>>> Synchronous training complete.


In [124]:
# df_transform_fin.show(5)

In [125]:
# df_transform_fin.printSchema()

In [126]:
# from pyspark.sql import Row
# from pyspark.ml.linalg import Vectors
# city_rdd = df_transform_fin.rdd.map(lambda row:row[3])
# temp_rdd = df_transform_fin.rdd.map(lambda row:row[0])
# new_df = city_rdd.zip(temp_rdd.map(lambda x:Vectors.dense(x))).toDF(schema=['label','features'])

# new_df

In [127]:
# new_df.show(5, truncate=False)

In [113]:
my_dl.write().overwrite().save('hdfs:///user/root/model_1')

In [89]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics

In [79]:
# pred_train = my_dl.transform(d1)
pred_test = my_dl.transform(d4)

In [80]:
pred_test.show(5)

+-----+--------------------+--------------------+-----------+--------------------+
|label|            feature1|            feature2|label_index|          prediction|
+-----+--------------------+--------------------+-----------+--------------------+
|    0|[638.0,617.0,1367...|[3741.0,3625.0,25...|        0.0|[0.99938523769378...|
|    0|[1604.0,968.0,143...|[1282.0,916.0,121...|        0.0|[0.99788296222686...|
|    0|[1447.0,338.0,239...|[416.0,837.0,416....|        0.0|[0.99794328212738...|
|    0|[479.0,369.0,3862...|[6.0,29.0,1511.0,...|        0.0|[0.99982774257659...|
|    0|[677.0,625.0,3556...|[4713.0,2203.0,49...|        0.0|[0.99977290630340...|
+-----+--------------------+--------------------+-----------+--------------------+
only showing top 5 rows



In [81]:
from pyspark.sql.functions import udf
list_to_double = udf(lambda l: l[0], DoubleType())
pred_test = pred_test.select(
    pred_test["label_index"], 
    list_to_double(pred_test["prediction"]).alias("prediction2")
)

In [83]:
pred_test.printSchema()

root
 |-- label_index: double (nullable = false)
 |-- prediction2: double (nullable = true)



In [84]:
pred_test.show(5, truncate=False)

+-----------+------------------+
|label_index|prediction2       |
+-----------+------------------+
|0.0        |0.9993852376937866|
|0.0        |0.9978829622268677|
|0.0        |0.9979432821273804|
|0.0        |0.9998277425765991|
|0.0        |0.9997729063034058|
+-----------+------------------+
only showing top 5 rows



In [85]:
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l:1.0 if l<0.5 else 0.0 , DoubleType())
pred_test = pred_test.select(
    pred_test["label_index"], 
    list_to_vector_udf(pred_test["prediction2"]).alias("rawPrediction")
)

In [233]:
pred_test.show(10, truncate=False)

+-----------+-------------+
|label_index|rawPrediction|
+-----------+-------------+
|1.0        |1.0          |
|1.0        |1.0          |
|1.0        |1.0          |
|1.0        |1.0          |
|1.0        |1.0          |
|1.0        |1.0          |
|1.0        |1.0          |
|1.0        |0.0          |
|1.0        |1.0          |
|1.0        |1.0          |
+-----------+-------------+
only showing top 10 rows



In [86]:
pred_test.groupBy('rawPrediction').count().orderBy('count').show()

+-------------+-----+
|rawPrediction|count|
+-------------+-----+
|          1.0| 6939|
|          0.0|42939|
+-------------+-----+



In [69]:
from pyspark.sql.functions import isnan, when, count, col

pred_test.select([count(when(isnan(c), c)).alias(c) for c in pred_test.columns]).show()

+-----------+-------------+
|label_index|rawPrediction|
+-----------+-------------+
|          0|            0|
+-----------+-------------+



#### ROC AUC구하기

In [84]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Let's use the run-of-the-mill evaluator
evaluator = BinaryClassificationEvaluator(labelCol='label_index')

# We have only two choices: area under ROC and PR curves :-(
auroc = evaluator.evaluate(pred_test, {evaluator.metricName: "areaUnderROC"})
auprc = evaluator.evaluate(pred_test, {evaluator.metricName: "areaUnderPR"})
print("Area under ROC Curve: {:.4f}".format(auroc))
print("Area under PR Curve: {:.4f}".format(auprc))

Area under ROC Curve: 0.1467
Area under PR Curve: 0.3095


#### Precision, Recall 구하기

In [90]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="label_index", predictionCol="rawPrediction", metricName="accuracy")

In [91]:
accuracy = evaluator.evaluate(pred_test)
print(accuracy)

0.5523076306187097


In [176]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="label_index", predictionCol="rawPrediction", metricName="f1")

In [177]:
f1 = evaluator.evaluate(pred_test)
print(f1)

0.9184711292471107


In [178]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="label_index", predictionCol="rawPrediction", metricName="weightedPrecision")

In [179]:
weightedPrecision = evaluator.evaluate(pred_test)
print(weightedPrecision)

0.919118309422521


In [180]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="label_index", predictionCol="rawPrediction", metricName="weightedRecall")

In [181]:
weightedRecall = evaluator.evaluate(pred_test)
print(weightedRecall)

0.9185011427884038


In [72]:
# pnl_train = pred_train.select('label_index', "prediction2")
pnl_test = pred_test.select('label_index', "rawPrediction")

In [73]:
# pred_and_label_train = pnl_train.rdd.map(lambda row: (row['label_index'], row['prediction2']))
pred_and_label_test = pnl_test.rdd.map(lambda row: (row['label_index'],row['rawPrediction']))

In [74]:
# metrics_train = MulticlassMetrics(pred_and_label_train)
metrics_test = MulticlassMetrics(pred_and_label_test)

In [75]:
# print("Training Data Accuracy: {}".format(round(metrics_train.precision(), 4)))
# print("Training Data Confusion Matrix")
# display(pnl_train.crosstab('label_index', 'prediction').toPandas())

print("\nTest Data Accuracy: {}".format(metrics_test.precision()))
# print("Test Data Confusion Matrix")
# display(pnl_test.crosstab("label_index", "prediction").toPandas())

TypeError: Invalid argument, not a string or column: 0.9539476322226232 of type <class 'float'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

In [76]:
metrics_test.precision()

0.9539476322226232

In [29]:
word_counts_1.value.get('allow')

105727

#### Broadcast 변수 update

In [63]:
class BroadcastWrapper(object):
    def __init__(self, data, token_list):
        self.broadcast_var = data
#         self.last_updated_time = datetime.now()
        self.token_list = token_list
        
#     def is_should_be_updated(self, data):
#         cur_time = datetime.now()
#         diff_sec = (cur_time - self.last_updated_time).total_seconds()
#         return self.broadcast_var is None or diff_sec> 1
    
    def update_and_get_data(self, spark):
        a = self.broadcast_var.value
        self.broadcast_var.unpersist()
        for i in self.token_list:
            a[i] += 1
        new_data = a
        
        self.broadcast_var = spark.broadcast(new_data)
#         self.last_updated_time = datetime.now()
    
        return self.broadcast_var

In [64]:
import datetime
aa = sc.broadcast({'a':1, 'b':2, 'c':3})
broadcast_wrapper = BroadcastWrapper(aa,['a', 'c'] )

In [65]:
a = broadcast_wrapper.update_and_get_data(sc)

In [66]:
a.value

{'a': 2, 'b': 2, 'c': 4}

In [69]:
broadcast_wrapper.broadcast_var.value

{'a': 2, 'b': 2, 'c': 4}

In [73]:
conf = spark.sparkContext._conf.setAll([("spark.streaming.concurrentJobs", "2")])

spark = SparkSession \
            .builder \
            .appName("TwitterClassification") \
            .config(conf=conf) \
            .config("spark.streaming.concurrentJobs", "2") \
            .getOrCreate()

In [74]:
sc.getConf().getAll()

[('spark.eventLog.enabled', 'true'),
 ('spark.driver.memory', '4g'),
 ('spark.executor.id', 'driver'),
 ('spark.history.fs.logDirectory', 'file:///root/spark/eventLog'),
 ('spark.driver.host', 'kafka1'),
 ('spark.network.timeout', '3600s'),
 ('spark.app.name', 'TwitterClassification'),
 ('spark.executor.memory', '6g'),
 ('spark.serializer', 'org.apache.spark.serializer.KryoSerializer'),
 ('spark.streaming.concurrentJobs', '2'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.driver.port', '38277'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.id', 'local-1632550969090'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.default.parallelism', '20'),
 ('spark.submit.deployMode', 'client'),
 ('spark.eventLog.dir', 'file:///root/spark/eventLog'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.kryoserializer.buffer.max', '2047'),
 ('spark.master', 'local[9]')]

In [68]:
spark

In [ ]:
dl_pipeline = Pipeline(stages=[estimator])
model = dl_pipeline.fit(df.map())

In [ ]:
if __name__ == "__main__":
    spark = SparkSession \
            .builder \
            .appName("TwitterClassification") \
            .config("spark.streaming.concurrentJobs", "2")
            .getOrCreate()

    spark.sparkContext.setLogLevel("ERROR")
    
    kafka_df = spark.readStream.format("kafka").option("partition.assignment.strategy","range").option("kafka.bootstrap.servers", "kafka1:9092,kafka2:9092,kafka3:9093").option("subscribe", "test1,test2,test3").option("startingOffsets", "latest").load()
    kafka_df.printSchema()
    kafka_df1 = kafka_df.selectExpr("CAST(value AS STRING)", "timestamp")
    
    transaction_detail_schema = StructType() \
            .add("id", StringType()) \
            .add("text", StringType()) \
            .add("created_at", StringType())
    
    kafka_df2 = kafka_df1 \
            .select(from_json(col("value"), transaction_detail_schema).alias("transaction_detail"), "timestamp")

    kafka_df3 = kafka_df2.select("transaction_detail.*", "timestamp")
 
    udf_blob = udf(apply_blob, StringType())
    
    udf_nlkt = udf(nlkt_analysis, StringType())
    
    new_df = kafka_df3.withColumn("status", udf_nlkt("text"))
    new_df = new_df.drop("text")

    def write_mongo_row(df, epoch_id):
        mongoURL = "mongodb://117.17.189.6:27017/tweet.kafka_tweet"
        df.write.format("com.mongodb.spark.sql.DefaultSource").mode("append").option("uri", mongoURL).save()
        df.show()
        pass
    query = new_df.writeStream.trigger(processingTime='1 seconds').foreachBatch(write_mongo_row).start() 
    print('---------------------------------------------')
    query.awaitTermination()

    spark.stop()


In [147]:
d1f = pd.DataFrame([[[80, 0],[90, 1]],[[70,13],[80,34]]],columns=['aa','nn'])
d1f

,aa,nn
0,"[80, 0]","[90, 1]"
1,"[70, 13]","[80, 34]"


In [148]:
d1f['aa'].values

array([list([80, 0]), list([70, 13])], dtype=object)

In [149]:
cols = ['aa','nn']
def make_original(x) :
    print(x.values)
    tmp = []
    tmp.append(x.values)
    return tmp
d1f.insert(0, 'eee', d1f[cols].apply(lambda row: make_original(row), axis=1))

[list([80, 0]) list([90, 1])]
[list([70, 13]) list([80, 34])]


In [150]:
a = list([80, 0])

In [151]:
type(a)

list

In [152]:
a[0]

80

In [153]:
d1f['eee']

0      [[[80, 0], [90, 1]]]
1    [[[70, 13], [80, 34]]]
Name: eee, dtype: object

In [155]:
d1f['eee'].iloc[0][0][0]

[80, 0]

In [112]:
d1f

,eee,aa,nn
0,"[[[80, 0], [90, 1]]]","[80, 0]","[90, 1]"
1,"[[[70, 13], [80, 34]]]","[70, 13]","[80, 34]"


In [ ]:
################################ EX ############################################

In [106]:
import pyspark.sql.functions as F
from pyspark.sql.functions import arrays_zip
sentenceData = spark.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish Java could use case classes"),
    (1.0, "Logistic regression models are neat")
]

df.show(truncate=False)

+---+-----------------------------------+
|_1 |_2                                 |
+---+-----------------------------------+
|0.0|Hi I heard about Spark             |
|0.0|I wish Java could use case classes |
|1.0|Logistic regression models are neat|
+---+-----------------------------------+



In [107]:
list_data = ['a' ,'b', 'c']

df = df.select(collect_list(struct(F.col("*"))).alias("data"))
df.show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------+
|data                                                                                                                  |
+----------------------------------------------------------------------------------------------------------------------+
|[[0.0, Hi I heard about Spark], [0.0, I wish Java could use case classes], [1.0, Logistic regression models are neat]]|
+----------------------------------------------------------------------------------------------------------------------+



In [108]:
df = df.withColumn("list",F.array([F.lit(i) for i in list_data]))
df.show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------+---------+
|data                                                                                                                  |list     |
+----------------------------------------------------------------------------------------------------------------------+---------+
|[[0.0, I wish Java could use case classes], [1.0, Logistic regression models are neat], [0.0, Hi I heard about Spark]]|[a, b, c]|
+----------------------------------------------------------------------------------------------------------------------+---------+



In [109]:
df = df.select(F.explode(F.arrays_zip(F.col("data"),F.col("list"))).alias("full_data"))
df.show(truncate=False)

+-----------------------------------------------+
|full_data                                      |
+-----------------------------------------------+
|[[0.0, I wish Java could use case classes], a] |
|[[1.0, Logistic regression models are neat], b]|
|[[0.0, Hi I heard about Spark], c]             |
+-----------------------------------------------+



In [110]:
df = df.select(F.col("full_data.data.*"),F.col("full_data.list").alias("col3"))
df.show(truncate=False)

+---+-----------------------------------+----+
|_1 |_2                                 |col3|
+---+-----------------------------------+----+
|0.0|Hi I heard about Spark             |a   |
|0.0|I wish Java could use case classes |b   |
|1.0|Logistic regression models are neat|c   |
+---+-----------------------------------+----+



In [ ]:
#################### 해당 내용 list에도 적용가능한지 확인########################

In [42]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
hashingTF = HashingTF(inputCol="pos_t", outputCol="rawFeatures", numFeatures=100)
featurizedData = hashingTF.transform(r4)
featurizedData.show(5)

+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|_c0|                text|label|               pos_t|               neg_t|     filtered_words1|     filtered_words2|         rawFeatures|
+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|  0|threatmeter hacke...|    0|[threatmeter, hac...|[threatmeter, hac...|[threatmeter, hac...|[threatmeter, hac...|(100,[7,12,15,20,...|
|  1|first android mal...|    0|[first, android, ...|[first, android, ...|[first, android, ...|[first, android, ...|(100,[3,11,40,69,...|
|  2|adobe fixes six c...|    0|[adobe, fixes, si...|[adobe, fixes, si...|[adobe, fixes, si...|[adobe, fixes, si...|(100,[5,20,40,45,...|
|  3| scienceporn  in ...|    0|[scienceporn, in,...|[scienceporn, in,...|[scienceporn, vac...|[scienceporn, vac...|(100,[29,38,45,70...|
|  4|riskware hmoyfzb ...|    0|[r

In [43]:
featurizedData = featurizedData.toPandas()
featurizedData['rawFeatures'].value_counts()

(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)    169
(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

In [44]:
len(featurizedData['rawFeatures'].iloc[1])

100

In [ ]:
### rawFeatures input으로 사용해서 모델 돌려보기

In [18]:
from pyspark.ml.feature import CountVectorizer
stage_3_a = CountVectorizer(inputCol=stage_2_a.getOutputCol(), outputCol='word1', vocabSize=50, minDF=2.0)
stage_3_b = CountVectorizer(inputCol=stage_2_b.getOutputCol(), outputCol='word2', vocabSize=50, minDF=2.0)

In [19]:
pipeline = Pipeline(stages= [stage_1_a, stage_1_b, stage_2_a, stage_2_b, stage_3_a, stage_3_b])
pipelineFit = pipeline.fit(csi_pos_neg)

In [29]:
df_transform = pipelineFit.transform(csi_pos_neg)
dfdfdf = df_transform.toPandas()

In [30]:
type(dfdfdf)

pandas.core.frame.DataFrame

In [36]:
dfdfdf['word1'].value_counts()

(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)    84912
(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)     5534
(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)     3799
(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [35]:
len(dfdfdf['word1'].iloc[1])

50

In [21]:
from pyspark.ml.feature import CountVectorizer
stage_3_a = CountVectorizer(inputCol=stage_2_a.getOutputCol(), outputCol='word1', vocabSize=100, minDF=2.0)
stage_3_b = CountVectorizer(inputCol=stage_2_b.getOutputCol(), outputCol='word2', vocabSize=100, minDF=2.0)

In [26]:
model = stage_3_a.fit(r4)
r5 = model.transform(r4)
r5.show(5)

+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+---------------+
|_c0|                text|label|               pos_t|               neg_t|     filtered_words1|     filtered_words2|          word1|
+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+---------------+
|  0|threatmeter hacke...|    0|[threatmeter, hac...|[threatmeter, hac...|[threatmeter, hac...|[threatmeter, hac...| (50,[4],[1.0])|
|  1|first android mal...|    0|[first, android, ...|[first, android, ...|[first, android, ...|[first, android, ...|(50,[29],[1.0])|
|  2|adobe fixes six c...|    0|[adobe, fixes, si...|[adobe, fixes, si...|[adobe, fixes, si...|[adobe, fixes, si...|     (50,[],[])|
|  3| scienceporn  in ...|    0|[scienceporn, in,...|[scienceporn, in,...|[scienceporn, vac...|[scienceporn, vac...|     (50,[],[])|
|  4|riskware hmoyfzb ...|    0|[riskware, hmoyfz...|[riskware, hmoyf

In [27]:
model2 = stage_3_b.fit(r5)
r6 = model2.transform(r5)
r6.show(5)

+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+---------------+---------------+
|_c0|                text|label|               pos_t|               neg_t|     filtered_words1|     filtered_words2|          word1|          word2|
+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+---------------+---------------+
|  0|threatmeter hacke...|    0|[threatmeter, hac...|[threatmeter, hac...|[threatmeter, hac...|[threatmeter, hac...| (50,[4],[1.0])| (50,[4],[1.0])|
|  1|first android mal...|    0|[first, android, ...|[first, android, ...|[first, android, ...|[first, android, ...|(50,[29],[1.0])|(50,[29],[1.0])|
|  2|adobe fixes six c...|    0|[adobe, fixes, si...|[adobe, fixes, si...|[adobe, fixes, si...|[adobe, fixes, si...|     (50,[],[])|     (50,[],[])|
|  3| scienceporn  in ...|    0|[scienceporn, in,...|[scienceporn, in,...|[scienceporn, vac...|[sciencepor

In [28]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols= [stage_3_a.getOutputCol(), stage_3_b.getOutputCol()], outputCol= 'features')

In [34]:
r7 = vectorAssembler.transform(r6)
r7.show(5, truncate=False)

+---+--------------------------------------------------------------------------------------------------------------------+-----+-------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------+---------------+---------------+-----------------------+
|_c0|text                                                                                                                |label|pos_t                                                                                                                                |neg_t                                            

In [ ]:
#########################################################################################

In [37]:
from pyspark.ml.feature import PolynomialExpansion
from pyspark.ml.linalg import Vectors

df = spark.createDataFrame([
    (Vectors.dense([2.0, 1.0]),),
    (Vectors.dense([0.0, 0.0]),),
    (Vectors.dense([3.0, -1.0]),)
], ["features"])

polyExpansion = PolynomialExpansion(degree=3, inputCol="features", outputCol="polyFeatures")
polyDF = polyExpansion.transform(df)

polyDF.show(truncate=False)

+----------+------------------------------------------+
|features  |polyFeatures                              |
+----------+------------------------------------------+
|[2.0,1.0] |[2.0,4.0,8.0,1.0,2.0,4.0,1.0,2.0,1.0]     |
|[0.0,0.0] |[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]     |
|[3.0,-1.0]|[3.0,9.0,27.0,-1.0,-3.0,-9.0,1.0,3.0,-1.0]|
+----------+------------------------------------------+



In [39]:
from pyspark.ml.feature import StringIndexer

df = spark.createDataFrame(
    [(0, "a"), (1, "b"), (2, "c"), (3, "a"), (4, "a"), (5, "c")],
    ["id", "category"])

indexer = StringIndexer(inputCol="category", outputCol="categoryIndex")
indexed = indexer.fit(df).transform(df)
indexed.show()

+---+--------+-------------+
| id|category|categoryIndex|
+---+--------+-------------+
|  0|       a|          0.0|
|  1|       b|          2.0|
|  2|       c|          1.0|
|  3|       a|          0.0|
|  4|       a|          0.0|
|  5|       c|          1.0|
+---+--------+-------------+



In [ ]:
#######################################################################################

In [15]:
pipeline = Pipeline(stages= [stage_1_a, stage_1_b, stage_2_a, stage_2_b, stage_3_a, stage_3_b, vectorAssembler])
pipelineFit = pipeline.fit(csi_pos_neg)

In [16]:
df_transform = pipelineFit.transform(csi_pos_neg)
df_transform.limit(5).toPandas()

,_c0,text,label,pos_t,neg_t,filtered_words1,filtered_words2,word1,word2,features
0,0,threatmeter hacked emails of san francisco mun...,0,"[threatmeter, hacked, emails, of, san, francis...","[threatmeter, hacked, emails, of, san, francis...","[threatmeter, hacked, emails, san, francisco, ...","[threatmeter, hacked, emails, san, francisco, ...","(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,first android malware targeting pcs uncovered ...,0,"[first, android, malware, targeting, pcs, unco...","[first, android, malware, targeting, pcs, unco...","[first, android, malware, targeting, pcs, unco...","[first, android, malware, targeting, pcs, unco...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,2,adobe fixes six code execution bugs in flash ...,0,"[adobe, fixes, six, code, execution, bugs, in,...","[adobe, fixes, six, code, execution, bugs, in,...","[adobe, fixes, six, code, execution, bugs, fla...","[adobe, fixes, six, code, execution, bugs, fla...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,3,scienceporn in a vacuum i guess,0,"[scienceporn, in, a, vacuum, i, guess]","[scienceporn, in, a, vacuum, i, guess]","[scienceporn, vacuum, guess]","[scienceporn, vacuum, guess]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,4,riskware hmoyfzb fbjaz,0,"[riskware, hmoyfzb, fbjaz]","[riskware, hmoyfzb, fbjaz]","[riskware, hmoyfzb, fbjaz]","[riskware, hmoyfzb, fbjaz]","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [17]:
df_transform_fin = df_transform.orderBy(rand())
nb_classes = df_transform_fin.distinct().count()

In [18]:
vocab_sizes = 5001
neg_vocab_sizes = 5001
embedding_vector_length = 100
hidden_dims = 250
max_length = 100

In [19]:
from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Input, Dense, concatenate
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers.embeddings import Embedding
from tensorflow.python.keras.layers import Conv1D, Flatten, GlobalMaxPooling1D, Dropout, MaxPooling1D, Activation, Bidirectional, Conv2D, GlobalMaxPooling2D

In [20]:
# CSI positive Embedding model
inputA = Input(shape=(100,))
x = Embedding(vocab_sizes, embedding_vector_length, input_length=max_length)(inputA)
# x = Conv1D(128, 5, activation='relu')(x)
# x = MaxPool1D(strides=1, padding='valid')(x)
x = Bidirectional(LSTM(128))(x)
x = Dropout(0.2)(x)
x = Activation('relu')(x)
x = Model(inputs=inputA, outputs=x)

# CSI negative Embedding model
inputC = Input(shape=(100,))
k = Embedding(neg_vocab_sizes, embedding_vector_length, input_length=max_length)(inputC)
# k = Conv1D(128, 5, activation='relu')(k)
# k = MaxPool1D(strides=1, padding='valid')(k)
k = Bidirectional(LSTM(128))(k)
k = Dropout(0.2)(k)
k = Activation('relu')(k)
k = Model(inputs=inputC, outputs=k)

combined = concatenate([x.output, k.output], axis=1)
# z = Flatten()(combined)
z = Dense(8, activation='relu')(combined)
z = Dense(1, activation="sigmoid")(z)

model = Model(inputs=[x.input, k.input], outputs=z)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [21]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 100)     500100      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 100, 100)     500100      input_2[0][0]                    
____________________________________________________________________________________________

In [22]:
from keras import optimizers, regularizers
from tensorflow.keras import optimizers

optimizer_conf = optimizers.Adam(lr=0.01)
opt_conf = optimizers.serialize(optimizer_conf)

In [41]:
from tensorflow.keras.models import model_from_json
from elephas import spark_model, ml_model
from elephas.ml_model import ElephasEstimator

estimator = ElephasEstimator()
estimator.setFeaturesCol("features")
estimator.setLabelCol("label")
estimator.set_keras_model_config(model.to_json())
# estimator.set_categorical_labels(True)
estimator.set_nb_classes(nb_classes)
estimator.set_num_workers(2)
estimator.set_epochs(3)
estimator.set_batch_size(32)
estimator.set_verbosity(1)
estimator.set_validation_split(0.10)
estimator.set_optimizer_config(opt_conf)
estimator.set_mode("synchronous")
estimator.set_loss("binary_crossentropy")
# estimator.set_metrics(['acc'])

dl_pipeline = Pipeline(stages=[estimator])
my_dl = dl_pipeline.fit(df_transform_fin)

ElephasEstimator_907d50e28eff

In [24]:
dl_pipeline = Pipeline(stages=[estimator])

In [25]:
my_dl = dl_pipeline.fit(df_transform_fin)

>>> Fit model


KeyboardInterrupt: 

In [ ]:
my_dl.write().overwrite().save('hdfs:///user/root/model_2')